On s’intéresse aux processus et à leur ordonnancement au sein d'un système d'exploitation.
On considère ici qu’on utilise un monoprocesseur.

1. Citer les 3 états dans lesquels un processus peut se trouver.

In [2]:
# NON DONNE DANS LE SUJET

class Processus:
    def __init__(self, nom: str, duree: int, priorite: int = 0):
        self.nom = nom
        self.duree = duree
        self.priorite = priorite
        self.reste = duree

    def execute_un_cycle(self):
        self.reste -= 1

    def est_fini(self) -> bool:
        return self.reste == 0



On veut simuler cet ordonnancement avec des objets.
Pour ce faire, on dispose déjà de la classe suivante, dont voici la documentation :

Classe Processus:
```
p = Processus(nom: str, duree: int)
    Crée un processus de nom <nom> et de durée <duree> (exprimée en cycles d'ordonnancement)

p.execute_un_cycle()
    Exécute le processus donné pendant un cycle.

p.est_fini()
    Renvoie True si le processus est terminé, False sinon.
```
Pour simplifier, on ne s'intéresse pas aux ressources qu'un processus pourrait acquérir ou libérer.

2. Citer les seuls 2 états possibles pour un processus dans ce contexte.

Pour mettre en place l'ordonnancement, on décide d'utiliser une file, instance de la classe `File` ci-dessous.

In [3]:
class File:
    def __init__(self):
        """ Renvoie une file vide """
        self.contenu = []

    def enfile(self, element):
        """ Enfile element dans la file """
        self.contenu.append(element)

    def defile(self):
        """ Renvoie le premier élément de la file """
        return self.contenu.pop(0)

    def est_vide(self):
        """ Renvoie True si la file est vide, False sinon """
        return self.contenu == []

Lors de la phase de tests, on se rend compte que le code suivant produit une erreur :

```
f = File()
print(f.defile())
```

3. Indiquer sur votre copie le code de la méthode `defile` rectifié pour que le code précédent ne produise plus d'erreur.

On se propose d'ordonnancer les processus avec une méthode du type *tourniquet*. À chaque cycle ;
- si un nouveau processus est créé, il est mis dans la file d'attente ;
- ensuite, on défile un processus de la file d'attente et on l'exécute pendant un cycle ;
- si le processus exécuté n'est pas terminé, on le replace dans la file.

Par exemple, avec les processus suivants

| processus | cycle de création | durée en cycles |
|:---------:|:-----------------:|:---------------:|
|    p1     |         2         |        3        |
|    p2     |         1         |        4        |
|    p3     |         4         |        3        |
|    p1     |         0         |        5        |

On obtient le chronogramme ci-dessous :

![chronogramme](img/00.png)

Pour décrire les processus et le moment de leur création, on utilise le code suivant, dans lequel `depart_proc` associe à un cycle donné le processus qui sera créé à ce moment :

In [4]:
p1 = Processus("p1", 4)
p2 = Processus("p2", 3)
p3 = Processus("p3", 5)
p4 = Processus("p4", 3)

depart_proc = {0: p1, 2: p2, 1: p3, 3: p4}

4. Compléter le chronogramme suivant

![chronogramme](img/01.png)

In [5]:
# NON DONNE DANS LE SUJET

class Ordonnanceur:

    def __init__(self):
        self.temps = 0
        self.file = File()

    def ajoute_nouveau_processus(self, proc: Processus):
        self.file.enfile(proc)

    def tourniquet(self) -> str | None:
        self.temps += 1
        if not self.file.est_vide():
            proc: Processus = self.file.defile()
            proc.execute_un_cycle()
            if not proc.est_fini():
                self.file.enfile(proc)
            return proc.nom

Pour mettre en place l'ordonnancement suivant cette méthode, on écrit la classe `Ordonnanceur` dont voici un code incomplet :

```
class Ordonnanceur:

    def __init__(self):
        self.temps = 0
        self.file = File()

    def ajoute_nouveau_processus(self, proc: Processus):
        ...

    def tourniquet(self) -> str | None:
        self.temps = ...
        if not self.file.est_vide():
            proc = ...
            ...
            if not proc.est_fini():
                ...
            return proc.nom
        else:
            return None
```

5. Compléter le code ci-dessus.

À chaque appel de la méthode `tourniquet` celle-ci renvoie ou bien le nom du processus qui a été élu, ou bien `None` s'il n'a pas trouvé de processus en cours.

6. Écrire un programme qui
- utilise les variables `p1`, `p2`, `p3`, `p4` et `depart_proc` définies ci-dessus ;
- crée un ordonnanceur ;
- ajoute un nouveau processus à l'ordonnanceur lorsque c'est le moment ;
- affiche le processus choisi par l'ordonnanceur ;
- s'arrête lorsqu'il n'y a plus de processus à exécuter.

In [6]:
ordonnanceur = Ordonnanceur()

fini = False
while not fini:
    if ordonnanceur.temps in depart_proc:
        ordonnanceur.ajoute_nouveau_processus(depart_proc[ordonnanceur.temps])
    elu = ordonnanceur.tourniquet()
    if elu:
        print(elu, end=" ")
    else:
        fini = True


p1 p1 p3 p1 p2 p3 p4 p1 p2 p3 p4 p2 p3 p4 p3 

Dans la situation donnée en exemple, il s'avère qu'en fait, les processus utilisent des ressources :
- un fichier commun aux processus ;
- le clavier de l'ordinateur ;
- le processeur graphique (GPU) ;
- le port 25000 de la connexion Internet.

Voici le détail de ce que fait chaque processus :

|         p1         |          p2          |        p3         |         p4          |
|:------------------:|:--------------------:|:-----------------:|:-------------------:|
|  acquérir le GPU   | acquérir le  clavier | acquerir le port  | acquérir le fichier |
| faire des calculs  |  aquérir le fichier  | faire des calculs |  faire des calculs  |
|   libérer le GPU   | libérer le clavier   | libérer le port   | acquérir le clavier |
|                    |  libérer le fichier  |                   | libérer le clavier  |
|                    |                      |                   | libérer le fichier  |

7. Montrer que l'ordre d'exécution donné en exemple aboutit à une situation d'interblocage.